In [ ]:
#This step focuses on understanding the problem, objects, constraints and expected outcomes
#Buisness Problem: Predict credit risk using financial features from the credit score dataset to identify individuals who are likely to default on loans
#Goal: Build a supervised classification model that accurately predicts whether a customer will default (1) or not default (0) based on their financial and demographic features
#Usefulness: Enables banks and lending institutions to identify high-risk individuals, optimise credit allocation, reduce financial losses and improve overall decision-making transparency. This helps lenders make informed, data-driven decisions to minimise financial risk.
#Task type: Supervised learning for binary classification (default vs non-default)

Step 2 - Data Understanding

In [ ]:
#In this step, you explore the dataset, check structure, inspect missing values and gain initial insights